In [1]:
import time
from getpass import getpass                                       # For logging in to Twitter through Python
from selenium import webdriver                                    # Our WebDriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

from selenium.webdriver.common.by import By                       # For Crawling
from selenium.webdriver.common.keys import Keys                   # For Crawling
from selenium.webdriver.chrome.options import ( 
    Options,) 
from selenium.common.exceptions import NoSuchElementException     # Avoiding adds
from selenium.webdriver.support import expected_conditions as EC  # Conditions
from selenium.webdriver.support.ui import (
    WebDriverWait,
) 

In [2]:
# Path configurations:
DRIVER_PATH = "C:\\Program Files (x86)\\chromedriver.exe"

# Set some options: default
options = Options()

# Start driver:
driver = webdriver.Chrome(service=Service(DRIVER_PATH), options=options)

In [ ]:
# Open Twitter (X) login page
web_site = "https://x.com/i/flow/login"
driver.get(web_site)

# Prompt the user for credentials
my_username = input("Provide a username: ")
my_password = getpass("Enter your password: ")

try:
    # Step 1: Enter username
    username_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@autocomplete='username']")) )
    username_field.send_keys(my_username)

    # Click the "Next" button
    next_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Next']")) )
    next_button.click()

    # Step 2: Enter password
    password_field = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@autocomplete='current-password']")) )
    password_field.send_keys(my_password)

    # Click the "Log in" button
    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "//span[text()='Log in']")) )
    login_button.click()

    # Wait for the home page to load
    WebDriverWait(driver, 10).until(
        EC.url_contains("/home") )
    print("Login successful!")

    # Step 3: Perform an advanced search for "enflasyon"
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//input[@aria-label='Search query']")) )
    search_box.clear()
    search_box.send_keys(
        '"enflasyon" lang:tr since:2024-01-01 -filter:links -filter:replies' )
    search_box.send_keys(Keys.RETURN)

    # Wait for the search results to load
    time.sleep(5)

    # Step 4: Scroll to load more tweets
    tweet_data = [] 
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3) 

        tweets = driver.find_elements(By.XPATH, "//article[@data-testid='tweet']")
        for tweet in tweets:
            try:
                # Extract tweet text
                tweet_text = tweet.find_element(By.XPATH, ".//div[@lang]").text

                # Extract tweet date
                tweet_date = tweet.find_element(By.XPATH, ".//time").get_attribute("datetime")

                # Append tweet text and date to the list
                tweet_data.append({"text": tweet_text, "date": tweet_date})
                
            except Exception as e:
                print("Error extracting tweet:", e)  # Print errors

        # Check if we've reached the end of the page
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # Exit if no more tweets are loading
        last_height = new_height

    # Step 5: Store tweets in a Pandas DataFrame
    df = pd.DataFrame(tweet_data)

    # Step 6: Export DataFrame to a pickle file
    df.to_pickle("enflasyon_tweets.pkl")

    print(f"Collected {len(df)} tweets about 'enflasyon'.")
    print("Tweets saved to 'enflasyon_tweets.pkl'.")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Close the browser
    driver.quit()

An error occurred: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=134.0.6998.89)
Stacktrace:
	GetHandleVerifier [0x00007FF603BEFE45+26629]
	(No symbol) [0x00007FF603B56010]
	(No symbol) [0x00007FF6039E931A]
	(No symbol) [0x00007FF6039D4F65]
	(No symbol) [0x00007FF6039F9E04]
	(No symbol) [0x00007FF603A6F85F]
	(No symbol) [0x00007FF603A8FA55]
	(No symbol) [0x00007FF603A67883]
	(No symbol) [0x00007FF603A30550]
	(No symbol) [0x00007FF603A31803]
	GetHandleVerifier [0x00007FF603F472BD+3529853]
	GetHandleVerifier [0x00007FF603F5DA22+3621858]
	GetHandleVerifier [0x00007FF603F524D3+3575443]
	GetHandleVerifier [0x00007FF603CBB77A+860474]
	(No symbol) [0x00007FF603B6088F]
	(No symbol) [0x00007FF603B5CBC4]
	(No symbol) [0x00007FF603B5CD66]
	(No symbol) [0x00007FF603B4C2C9]
	BaseThreadInitThunk [0x00007FF9799FE8D7+23]
	RtlUserThreadStart [0x00007FF97A5BBF2C+44]

